In [1]:
from models import VariationalAutoencoder, Autoencoder, ConvAutoencoder, ConvVAE, MyModel
from utils import train
from data_utils import CustomImageDataset
import torch
import os
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import tqdm

In [2]:
torch.manual_seed(0)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device:', device)

Using device: cuda


In [4]:
transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.CenterCrop(28),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0), std=(1)),
    #  transforms.RandomApply(
    #      [transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.)),
    #       transforms.RandomAdjustSharpness(sharpness_factor=2),
    #       transforms.RandomAutocontrast()]
    #  )
     ])

In [5]:
train_data_path = "./homework_2_dataset/dataset/train"
dataset = CustomImageDataset(os.path.join(train_data_path, "annotation.txt"), os.path.join(train_data_path, "imgs"), transform=transform)
train_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

### AE

In [6]:
ae = Autoencoder(16).to(device)
opt = torch.optim.Adam(ae.parameters())
for epoch in tqdm.tqdm(range(20)):
    running_loss = 0
    count = 0
    for x, y in train_dataloader:
        count += 1
        x = x.to(device) # GPU
        opt.zero_grad()
        x_hat = ae(x)
        loss = ((x - x_hat)**2).sum()
        running_loss += loss
        loss.backward()
        opt.step()
    print(f"epoch {epoch} loss: {loss}")

  5%|▌         | 1/20 [00:03<01:01,  3.23s/it]

epoch 0 loss: 74.70916748046875


 10%|█         | 2/20 [00:05<00:53,  2.95s/it]

epoch 1 loss: 32.499534606933594


 15%|█▌        | 3/20 [00:08<00:48,  2.85s/it]

...

epoch 17 loss: 10.544677734375


 95%|█████████▌| 19/20 [00:52<00:02,  2.74s/it]

epoch 18 loss: 13.595787048339844


100%|██████████| 20/20 [00:55<00:00,  2.78s/it]

epoch 19 loss: 13.811305046081543


In [7]:
torch.save(ae.state_dict(), "./autoencoders/AE")

### VAE

In [8]:
vae = VariationalAutoencoder(16).to(device) # GPU
opt = torch.optim.Adam(vae.parameters())
for epoch in tqdm.tqdm(range(20)):
    running_loss = 0
    count = 0
    for x, y in train_dataloader:
        count += 1
        x = x.to(device) # GPU
        opt.zero_grad()
        x_hat = vae(x)

        loss = ((x - x_hat)**2).sum() + vae.encoder.kl

        running_loss += loss
        loss.backward()
        opt.step()
    print(f"epoch {epoch} loss: {loss}")

  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:02<00:54,  2.88s/it]

epoch 0 loss: 234.82110595703125


 10%|█         | 2/20 [00:05<00:52,  2.94s/it]

epoch 1 loss: 228.84225463867188


...

epoch 17 loss: 176.25448608398438


 95%|█████████▌| 19/20 [00:55<00:02,  2.94s/it]

epoch 18 loss: 173.1685791015625


100%|██████████| 20/20 [00:58<00:00,  2.92s/it]

epoch 19 loss: 195.20933532714844


In [9]:
torch.save(vae.state_dict(), "./autoencoders/VAE")

### convolutional AE

In [10]:
conv = ConvAutoencoder().to(device) # GPU
opt = torch.optim.Adam(conv.parameters())
for epoch in tqdm.tqdm(range(20)):
    running_loss = 0
    count = 0
    for x, y in train_dataloader:
        count += 1
        x = x.to(device) # GPU
        opt.zero_grad()
        x_hat = conv(x)

        loss = ((x - x_hat)**2).sum()
        running_loss += loss
        loss.backward()
        opt.step()
    print(f"epoch {epoch} loss: {loss}")

  5%|▌         | 1/20 [00:03<00:57,  3.01s/it]

epoch 0 loss: 338.4342956542969


 10%|█         | 2/20 [00:05<00:51,  2.87s/it]

epoch 1 loss: 103.78974914550781


 15%|█▌        | 3/20 [00:08<00:47,  2.81s/it]

...

epoch 17 loss: 17.69014549255371


 95%|█████████▌| 19/20 [00:52<00:02,  2.74s/it]

epoch 18 loss: 14.752422332763672


100%|██████████| 20/20 [00:54<00:00,  2.75s/it]

epoch 19 loss: 19.873538970947266


In [11]:
torch.save(conv.state_dict(), "./autoencoders/conv_ae")

### convolutional VAE

In [12]:
conv_vae = ConvVAE().to(device) # GPU

In [13]:
opt = torch.optim.Adam(conv_vae.parameters())
for epoch in tqdm.tqdm(range(5)):
    running_loss = 0
    count = 0
    for x, y in train_dataloader:
        count += 1
        x = x.to(device) # GPU
        opt.zero_grad()
        reconstruction, mu, logvar = conv_vae(x)
        loss = conv_vae.loss(reconstruction, mu, logvar, x)
        running_loss += loss.item()
        loss.backward()
        opt.step()
    print(f"epoch {epoch} loss: {loss}")

 20%|██        | 1/5 [00:03<00:14,  3.66s/it]

epoch 0 loss: 0.5948857069015503


 40%|████      | 2/5 [00:07<00:10,  3.52s/it]

epoch 1 loss: 0.1543664038181305


 60%|██████    | 3/5 [00:10<00:06,  3.49s/it]

epoch 2 loss: 0.09884746372699738


 80%|████████  | 4/5 [00:14<00:03,  3.49s/it]

epoch 3 loss: 0.08305766433477402


100%|██████████| 5/5 [00:17<00:00,  3.50s/it]

epoch 4 loss: 0.0948648452758789


In [14]:
torch.save(conv_vae.state_dict(), "./autoencoders/conv_vae")

### MyModel

In [6]:
mm = MyModel().to(device) # GPU

In [7]:
opt = torch.optim.Adam(mm.parameters())
for epoch in tqdm.tqdm(range(10)):
    running_loss = 0
    count = 0
    for x, y in train_dataloader:
        count += 1
        x = x.to(device) # GPU
        opt.zero_grad()
        reconstruction, mu, logvar = mm(x)
        loss = mm.loss(reconstruction, x)
        running_loss += loss.item()
        loss.backward()
        opt.step()
    print(f"epoch {epoch} loss: {loss}")

 10%|█         | 1/10 [00:03<00:28,  3.19s/it]

epoch 0 loss: 0.0011297822929918766


 20%|██        | 2/10 [00:06<00:24,  3.07s/it]

epoch 1 loss: 0.0008167898049578071


 30%|███       | 3/10 [00:09<00:21,  3.03s/it]

...

epoch 7 loss: 0.00028343108715489507


 90%|█████████ | 9/10 [00:26<00:02,  2.87s/it]

epoch 8 loss: 0.0002847459109034389


100%|██████████| 10/10 [00:29<00:00,  2.94s/it]

epoch 9 loss: 0.00032429469865746796


In [8]:
torch.save(mm.state_dict(), "./autoencoders/my_model")

In [9]:
# mm2 = MyModel().to(device) # GPU
# opt = torch.optim.Adam(mm2.parameters())
# for epoch in tqdm.tqdm(range(10)):
#     running_loss = 0
#     count = 0
#     for x, y in train_dataloader:
#         count += 1
#         x = x.to(device) # GPU
#         opt.zero_grad()
#         reconstruction, mu, logvar = mm2(x)
#         loss = mm2.loss(reconstruction, x)
#         running_loss += loss.item()
#         loss.backward()
#         opt.step()
#     print(f"epoch {epoch} loss: {loss}")

# torch.save(mm2.state_dict(), "./autoencoders/my_model2")

 10%|█         | 1/10 [00:02<00:26,  2.89s/it]

epoch 0 loss: 0.0022954412270337343


 20%|██        | 2/10 [00:05<00:22,  2.81s/it]

epoch 1 loss: 0.0009621730423532426


 30%|███       | 3/10 [00:08<00:19,  2.81s/it]

...

epoch 7 loss: 0.00036637450102716684


 90%|█████████ | 9/10 [00:25<00:02,  2.85s/it]

epoch 8 loss: 0.00036983960308134556


100%|██████████| 10/10 [00:28<00:00,  2.85s/it]

epoch 9 loss: 0.0003258824872318655
